# Hourglass module

In [13]:
import torch
import numpy as np
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)

import warnings
warnings.filterwarnings('ignore')

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, num_channels=256):
        super(ResidualBlock, self).__init__()

        self.bn1 = nn.BatchNorm2d(num_channels)
        self.conv1 = nn.Conv2d(num_channels, num_channels//2, kernel_size=1, bias=True)

        self.bn2 = nn.BatchNorm2d(num_channels//2)
        self.conv2 = nn.Conv2d(num_channels//2, num_channels//2, kernel_size=3, stride=1, padding=1, bias=True)

        self.bn3 = nn.BatchNorm2d(num_channels//2)
        self.conv3 = nn.Conv2d(num_channels//2, num_channels, kernel_size=1, bias=True)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)

        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)

        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)

        out += residual

        return out

In [15]:
class Hourglass(nn.Module):
    def __init__(self, block, num_channels=256):
        super(Hourglass, self).__init__()

        self.downconv_1 = block(num_channels)
        self.pool_1 = nn.MaxPool2d(kernel_size=2)
        self.downconv_2 = block(num_channels)
        self.pool_2 = nn.MaxPool2d(kernel_size=2)
        self.downconv_3 = block(num_channels)
        self.pool_3 = nn.MaxPool2d(kernel_size=2)
        self.downconv_4 = block(num_channels)
        self.pool_4 = nn.MaxPool2d(kernel_size=2)

        self.midconv_1 = block(num_channels)
        self.midconv_2 = block(num_channels)
        self.midconv_3 = block(num_channels)
        
        self.skipconv_1 = block(num_channels)
        self.skipconv_2 = block(num_channels)
        self.skipconv_3 = block(num_channels)
        self.skipconv_4 = block(num_channels)

        self.upconv_1 = block(num_channels)
        self.upconv_2 = block(num_channels)
        self.upconv_3 = block(num_channels)
        self.upconv_4 = block(num_channels)

    def forward(self, x):

        x1 = self.downconv_1(x)
        x  = self.pool_1(x1)
        x2 = self.downconv_2(x)
        x  = self.pool_2(x2)
        x3 = self.downconv_3(x)
        x  = self.pool_3(x3)
        x4 = self.downconv_4(x)
        x  = self.pool_4(x4)

        x = self.midconv_1(x)
        x = self.midconv_2(x)
        x = self.midconv_3(x)

        x4 = self.skipconv_1(x4)
        x = F.upsample(x, scale_factor=2)
        x = x + x4
        x = self.upconv_1(x)

        x3 = self.skipconv_1(x3)
        x = F.upsample(x, scale_factor=2)
        x = x + x3
        x = self.upconv_2(x)

        x2 = self.skipconv_1(x2)
        x = F.upsample(x, scale_factor=2)
        x = x + x2
        x = self.upconv_3(x)

        x1 = self.skipconv_1(x1)
        x = F.upsample(x, scale_factor=2)
        x = x + x1
        x = self.upconv_4(x)

        return x

In [16]:
hg = Hourglass(ResidualBlock)

from torchsummary import summary
summary(hg, input_size=(256,64,64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1          [-1, 256, 64, 64]             512
              ReLU-2          [-1, 256, 64, 64]               0
            Conv2d-3          [-1, 128, 64, 64]          32,896
       BatchNorm2d-4          [-1, 128, 64, 64]             256
              ReLU-5          [-1, 128, 64, 64]               0
            Conv2d-6          [-1, 128, 64, 64]         147,584
       BatchNorm2d-7          [-1, 128, 64, 64]             256
              ReLU-8          [-1, 128, 64, 64]               0
            Conv2d-9          [-1, 256, 64, 64]          33,024
    ResidualBlock-10          [-1, 256, 64, 64]               0
        MaxPool2d-11          [-1, 256, 32, 32]               0
      BatchNorm2d-12          [-1, 256, 32, 32]             512
             ReLU-13          [-1, 256, 32, 32]               0
           Conv2d-14          [-1, 128,

# Stacked hourglass network

In [17]:
__all__ = ['HourglassNet', 'hg']

class Bottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()

        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn3 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes*2, kernel_size=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)

        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)

        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out


class Hourglass(nn.Module):
    def __init__(self, block, num_blocks, planes, depth):
        super(Hourglass, self).__init__()
        self.depth = depth
        self.block = block
        self.hg = self._make_hour_glass(block, num_blocks, planes, depth)

    def _make_residual(self, block, num_blocks, planes):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(planes*block.expansion, planes))
        return nn.Sequential(*layers)

    def _make_hour_glass(self, block, num_blocks, planes, depth):
        hg = []
        for i in range(depth):
            res = []
            for j in range(3):
                res.append(self._make_residual(block, num_blocks, planes))
            if i == 0:
                res.append(self._make_residual(block, num_blocks, planes))
            hg.append(nn.ModuleList(res))
        return nn.ModuleList(hg)

    def _hour_glass_forward(self, n, x):
        up1 = self.hg[n-1][0](x)
        low1 = F.max_pool2d(x, 2, stride=2)
        low1 = self.hg[n-1][1](low1)

        if n > 1:
            low2 = self._hour_glass_forward(n-1, low1)
        else:
            low2 = self.hg[n-1][3](low1)
        low3 = self.hg[n-1][2](low2)
        up2 = F.interpolate(low3, scale_factor=2)
        out = up1 + up2
        return out

    def forward(self, x):
        return self._hour_glass_forward(self.depth, x)


class HourglassNet(nn.Module):
    def __init__(self, block, num_stacks=2, num_blocks=4, num_classes=16):
        super(HourglassNet, self).__init__()

        self.inplanes = 64
        self.num_feats = 128
        self.num_stacks = num_stacks
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=True)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_residual(block, self.inplanes, 1)
        self.layer2 = self._make_residual(block, self.inplanes, 1)
        self.layer3 = self._make_residual(block, self.num_feats, 1)
        self.maxpool = nn.MaxPool2d(2, stride=2)

        ch = self.num_feats*block.expansion
        hg, res, fc, score, fc_, score_ = [], [], [], [], [], []
        for i in range(num_stacks):
            # Hourglass
            hg.append(Hourglass(block, num_blocks, self.num_feats, 4))
            # _make_residual
            res.append(self._make_residual(block, self.num_feats, num_blocks))
            # _make_fc
            fc.append(self._make_fc(ch, ch))
            score.append(nn.Conv2d(ch, num_classes, kernel_size=1, bias=True))
            if i < num_stacks-1:
                fc_.append(nn.Conv2d(ch, ch, kernel_size=1, bias=True))
                score_.append(nn.Conv2d(num_classes, ch, kernel_size=1, bias=True))
        self.hg = nn.ModuleList(hg)
        self.res = nn.ModuleList(res)
        self.fc = nn.ModuleList(fc)
        self.score = nn.ModuleList(score)
        self.fc_ = nn.ModuleList(fc_)
        self.score_ = nn.ModuleList(score_)

    def _make_residual(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes*block.expansion:
            downsample = nn.Sequential(nn.Conv2d(self.inplanes, planes*block.expansion, kernel_size=1, stride=stride, bias=True))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes*block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _make_fc(self, inplanes, outplanes):
        bn = nn.BatchNorm2d(inplanes)
        conv = nn.Conv2d(inplanes, outplanes, kernel_size=1, bias=True)
        return nn.Sequential(conv, bn, self.relu)

    def forward(self, x):
        out = []
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.maxpool(x)
        x = self.layer2(x)
        x = self.layer3(x)

        for i in range(self.num_stacks):
            y = self.hg[i](x)
            y = self.res[i](y)
            y = self.fc[i](y)
            score = self.score[i](y)
            out.append(score)
            if i < self.num_stacks-1:
                fc_ = self.fc_[i](y)
                score_ = self.score_[i](score)
                x = x + fc_ + score_

        return out

In [18]:
model = HourglassNet(Bottleneck, num_stacks=1, num_blocks=2, num_classes=22)

summary(model, input_size=(3,64,64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,472
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
              ReLU-4           [-1, 64, 32, 32]               0
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]           4,160
       BatchNorm2d-8           [-1, 64, 32, 32]             128
              ReLU-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,928
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 32, 32]           8,320
           Conv2d-14          [-1, 128,